# `PIVOT` og `UNPIVOT`

> Udviklet af Thomas Lange & Mick Ahlmann Brun

Mere info: [https://github.com/M1ckB/T-SQL](https://github.com/M1ckB/T-SQL)

Version 1.0 2023-01-17

Laboratoriet kræver:

- En understøttet version af SQL Server
- En Stack Overflow database: [Brent Ozar](https://www.BrentOzar.com/go/querystack) (medium)

Læs mere om `PIVOT` og `UNPIVOT` i Microsofts T-SQL reference:

- [https://learn.microsoft.com/en-us/sql/t-sql/queries/from-using-pivot-and-unpivot?view=sql-server-ver16](https://learn.microsoft.com/en-us/sql/t-sql/queries/from-using-pivot-and-unpivot?view=sql-server-ver16)


## Indholdsfortegnelse

- [Pivotering](#Pivotering)
  - [PIVOT-operatoren](#PIVOT-operatoren)
  - [UNPIVOT-operatoren](#UNPIVOT-operatoren)
- [Hovedpointer](#Hovedpointer)


## Pivotering

I T-SQL eksisterer der nogle ekstra tabeloperatorer ud over `JOIN`.

Der er implementeret to tabeloperatorer, `PIVOT` og `UNPIVOT`, til at pivotere data fra rækker til kolonner og, omvendt, fra kolonner til rækker.

Use cases:

- Pivotering til præsentationsformål, her bruges særligt `PIVOT` (resultatet bliver til en "crosstab")
- Normalisering af en tabel, her bruges særligt `UNPIVOT`

De to operatorer vil gennemgås nedenfor.


## `PIVOT`-operatoren

**Fra rækker til kolonner!**

`PIVOT`-operationer af data handler om at rotere data fra rækker til kolonner, eventuelt med samtidig aggregering af værdier.

En almindelig use case er, at man ønsker at rapportere data for flere år på en enkelt linje.

`PIVOT` vender et tabel-resultatet fra en `SELECT`-forespørgsel ved at forvandle unikke værdier fra en kolonne (i flere rækker) til flere kolonner.

`PIVOT` foretager aggregeringer hvor nødvendigt på tilbageværende kolonneværdier.

Pivotering handler altid om at identificere involverede elementer:

- Grupperingselementet
- Spredningselementet
- Aggregeringselementet
- Aggregeringsfunktionen


In [ ]:
/* Nedenstående er et eksempel på brug af PIVOT */

CREATE TABLE #OrigTxt (
  GroupId int NOT NULL,
  SpreadCol nvarchar(1) NOT NULL, 
  AggCol int NOT NULL
);
INSERT INTO #OrigTxt (GroupId, SpreadCol, AggCol)
VALUES
(1, 'A', 100), 
(2, 'A', 200), 
(2, 'B', 300);

CREATE TABLE #OrigNum (
  GroupId int NOT NULL,
  SpreadCol int NOT NULL, 
  AggCol int NOT NULL
);
INSERT INTO #OrigNum (GroupId, SpreadCol, AggCol)
VALUES
(1, 10, 100), 
(2, 10, 200), 
(2, 20, 300);

SELECT * FROM #OrigTxt;
SELECT * FROM #OrigNum;

/*
Hvad er grupperingselementet? Id
Hvad er spredningselementet? GrpCol
Hvad er aggregeringselementet? SumCol
Hvad er aggregeringsfunktionen? SUM()
*/

/* Hvis vi skulle gøre det 'i hånden' ... */
--med tekst spredningskolonne
SELECT GroupId, 
 SUM(CASE WHEN SpreadCol = 'A' THEN AggCol END) AS A, 
 SUM(CASE WHEN SpreadCol = 'B' THEN AggCol END) AS B
FROM #OrigTxt 
GROUP BY GroupId;

--med numerisk spredningskolonne.
SELECT GroupId, 
 SUM(CASE WHEN SpreadCol = 10 THEN AggCol END) AS [10], 
 SUM(CASE WHEN SpreadCol = 20 THEN AggCol END) AS [20]
FROM #OrigNum 
GROUP BY GroupId;

/* Med PIVOT bliver syntaksen mere kompakt og ens for tekst og numeriske spredningskolonner */

SELECT GroupId, [A], [B]
FROM #OrigTxt
PIVOT( SUM(AggCol) FOR SpreadCol IN ([A],[B]) ) AS p;

SELECT GroupId, [10], [20]
FROM #OrigNum
PIVOT( SUM(AggCol) FOR SpreadCol IN ([10],[20]) ) AS p;

DROP TABLE #OrigTxt;
DROP TABLE #OrigNum;


## `UNPIVOT`-operatoren

**Fra kolonner til rækker!**

`UNPIVOT`-operationen er en teknik, som roterer data fra kolonner til rækker.

En almindelig use case er, at man modtager data med flere kolonner for samme oplysning for flere år og ønsker en tabel, som muliggør filtrering på år.

`UNPIVOT` forvandler kolonner i tabel-resultatet fra en `SELECT`-forespørgsel til kolonneværdier (i flere rækker).


In [ ]:
/* Nedenstående er et eksempel på brug af UNPIVOT */

CREATE TABLE #TableA (
  Id int NOT NULL,
  [2020] nvarchar(1) NOT NULL, 
  [2021] nvarchar(1) NOT NULL, 
  [2022] nvarchar(1) NOT NULL);

INSERT INTO #TableA (Id, [2020], [2021], [2022])
VALUES
(1, 'A', 'B', 'C'), 
(2, 'B', 'C', 'D'), 
(3, 'E', 'F', 'G');

SELECT * FROM #TableA;

-- UNPIVOT  
SELECT Id, [Year], [Value]
FROM #TableA
UNPIVOT  ([Value] FOR [Year] IN ([2020], [2021], [2022])) up;  

DROP TABLE #TableA;


### _Tid til opgaver..._
Lav opgave 1, 2 og 3 i [opgavehæftet]().

## Hovedpointer

- HOVEDPOINTE


## Licens

Creative Commons Attribution-ShareAlike 4.0 International (CC BY-SA 4.0)

Mere info: [https://creativecommons.org/licenses/by-sa/4.0/](https://creativecommons.org/licenses/by-sa/4.0/)

Du kan frit:

- Dele: kopiere og distribuere materialet via ethvert medium og i ethvert format
- Tilpasse: remixe, redigere og bygge på materialet til ethvert formål, selv erhvervsmæssigt

Under følgende betingelser:

- Kreditering: Du skal kreditere, dele et link til licensen og indikere om der er lavet ændringer.
- Del på samme vilkår: Hvis du remixer, redigerer eller bygger på materialet, så skal dine bidrag
  distribueres under samme licens som den originale.
